In [39]:
import os
import numpy as np
import pandas as pd
import wfdb
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import seaborn as sns
from app.ecg.ecg import Datasets, EcgSignal    

ptbxl_data = pd.read_csv(r'C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\labels.csv', index_col=0)
# ptbxl_statements = pd.read_csv(Datasets.ptbxl_scp_statements.path, index_col=0)
# ptbxl_data['patient_id'] = ptbxl_data['patient_id'].astype(int)
# ptbxl_data['nurse'] = ptbxl_data['nurse'].astype('Int64')
# ptbxl_data['site'] = ptbxl_data['site'].astype('Int64')
# ptbxl_data['validated_by'] = ptbxl_data['validated_by'].astype('Int64')


In [2]:
ptbxl_data

,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,...,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,scp_codes_len,superdiagnostic,superdiagnostic_len
patient_id,,,,,,,,,,,,,,,,,,,,,
15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,3,['NORM'],1
13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,2,['NORM'],1
20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,2,['NORM'],1
17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,2,['NORM'],1
17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,2,['NORM'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",...,NaN,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr,4,['STTC'],1
20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr,3,['NORM'],1
19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,"{'ISCAS': 50.0, 'SR': 0.0}",...,NaN,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr,2,['STTC'],1


In [40]:

def get_file_paths(filename):
    filename_lr = filename.split('/')[-1].split('_')[0]
    return fr"C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\{filename_lr}.npy"

clean_tags = lambda x: [e.replace("'", "")  for e in x[1:-1].split(', ')]
ptbxl_data["file_paths"] = ptbxl_data["filename_lr"].apply(get_file_paths)
ptbxl_data["superdiagnostic"] = ptbxl_data["superdiagnostic"].apply(clean_tags)

In [12]:
ptbxl_data

,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,...,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,scp_codes_len,superdiagnostic,superdiagnostic_len,file_paths
patient_id,,,,,,,,,,,,,,,,,,,,,
15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",...,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr,4,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...
19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,"{'ISCAS': 50.0, 'SR': 0.0}",...,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr,2,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...


# Data preparation

In [41]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(ptbxl_data["superdiagnostic"].values)
mlb.classes_.tolist()

['CD', 'HYP', 'MI', 'NORM', 'STTC']

In [42]:
train_labels = mlb.transform(ptbxl_data["superdiagnostic"].tolist())
ptbxl_data[mlb.classes_.tolist()] = train_labels

In [43]:
ptbxl_data

,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,...,filename_hr,scp_codes_len,superdiagnostic,superdiagnostic_len,file_paths,CD,HYP,MI,NORM,STTC
patient_id,,,,,,,,,,,,,,,,,,,,,
15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",...,records500/00000/00001_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,records500/00000/00002_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,records500/00000/00003_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,records500/00000/00004_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,records500/00000/00005_hr,2,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...",...,records500/21000/21833_hr,4,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,0,1
20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}",...,records500/21000/21834_hr,3,[NORM],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,1,0
19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,"{'ISCAS': 50.0, 'SR': 0.0}",...,records500/21000/21835_hr,2,[STTC],1,C:\Users\redmi\PycharmProjects\ecg-tool-api\da...,0,0,0,0,1


In [44]:
ptbxl_data.columns

Index(['age', 'sex', 'height', 'weight', 'nurse', 'site', 'device',
       'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'scp_codes_len', 'superdiagnostic', 'superdiagnostic_len', 'file_paths',
       'CD', 'HYP', 'MI', 'NORM', 'STTC'],
      dtype='object')

In [45]:
from app.ecg.ecg import EcgDataset
dataset = EcgDataset(ptbxl_data)


In [47]:
ecg, label = dataset[0]
ecg

array([[-0.119, -0.055,  0.064, ..., -0.026, -0.039, -0.079],
       [-0.116, -0.051,  0.065, ..., -0.031, -0.034, -0.074],
       [-0.12 , -0.044,  0.076, ..., -0.028, -0.029, -0.069],
       ...,
       [ 0.069,  0.   , -0.069, ...,  0.024, -0.041, -0.058],
       [ 0.086,  0.004, -0.081, ...,  0.242, -0.046, -0.098],
       [ 0.022, -0.031, -0.054, ...,  0.143, -0.035, -0.12 ]])